In [1]:
cd ~/git/acoustic_activations_interp/

/home/ellenar/git/acoustic_activations_interp


In [2]:
import torch
from torch.utils.data import DataLoader

In [3]:
from activation_module import WhipserActivationModule
from dataset import ClasswiseDataset

/venv/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/venv/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [8]:
layer = 'encoder.blocks.0'
samples_per_class = 480_000
batch_size = int(samples_per_class / 480_000)
batch_size

1

### NON-SPEECH

In [9]:
module = WhipserActivationModule(activations_to_cache=[layer], data_class="NON_SPEECH", samples_per_class=samples_per_class, batch_size=batch_size)
module.forward()
non_speech_activations = module.activations[f'{layer}.output']
non_speech_activations = torch.reshape(non_speech_activations, (-1, 384))

Class label:NON_SPEECH, Number of Samples:480000
Decoding:torch.Size([1, 80, 3000])


/home/ellenar/git/acoustic_activations_interp/dataset.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mels = torch.tensor(whisper.log_mel_spectrogram(audio)).to(device)


### SPEECH

In [6]:
module = WhipserActivationModule(activations_to_cache=[layer], data_class="SPEECH", samples_per_class=samples_per_class, batch_size=batch_size)
module.forward()
speech_activations = module.activations[f'{layer}.output']
print(speech_activations.shape)
speech_activations = torch.reshape(speech_activations, (-1, 384))

Class label:SPEECH, Number of Samples:480000
Decoding:torch.Size([1, 80, 3000])


/home/ellenar/git/acoustic_activations_interp/dataset.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mels = torch.tensor(whisper.log_mel_spectrogram(audio)).to(device)


torch.Size([1, 1500, 1536])


### Compute dataset mean

In [8]:
all_activations = torch.cat((speech_activations, non_speech_activations), dim=0)
mean_activation = torch.mean(all_activations, dim=0)

### Normalize

In [9]:
speech_normed = (speech_activations / speech_activations.norm(dim=1)[:, None])
non_speech_normed = (non_speech_activations / non_speech_activations.norm(dim=1)[:, None])

In [10]:
# speech_normed = speech_activations - mean_activation
# speech_normed = (speech_normed / speech_normed.norm(dim=1)[:, None])
# non_speech_normed = non_speech_activations - mean_activation
# non_speech_normed = (non_speech_normed / non_speech_normed.norm(dim=1)[:, None])

### Get cosine sim scores

In [11]:
non_speech_sim = torch.mm(non_speech_normed, non_speech_normed.transpose(0,1))
print('Non speech', torch.mean(non_speech_sim))
speech_sim = torch.mm(speech_normed, speech_normed.transpose(0,1))
torch.mean(speech_sim)
print('Speech', torch.mean(speech_sim))
inter_class_sim = torch.mm(speech_normed, non_speech_normed.transpose(0,1))
print('Inter', torch.mean(inter_class_sim))

Non speech tensor(0.1542)
Speech tensor(0.1562)
Inter tensor(0.1541)


### Get euclidean scores

In [47]:
non_speech_dist = torch.cdist(non_speech_activations, non_speech_activations, p=2.0)
speech_dist = torch.cdist(speech_activations, speech_activations, p=2.0)
inter_dist = torch.cdist(speech_activations, non_speech_activations, p=2.0)

In [ ]:
print(torch.mean(speech_dist), torch.mean(non_speech_dist), torch.mean(inter_dist))

### Compute SVDs

In [ ]:
# TODO: check SVD is along correct dim
import numpy as np
U, S, Vh = np.linalg.svd(speech_normed, full_matrices=False, compute_uv=True, hermitian=False)
S

In [ ]:
U, S, Vh = np.linalg.svd(non_speech_normed, full_matrices=False, compute_uv=True, hermitian=False)
S

In [ ]:
U, S, Vh = np.linalg.svd(torch.cat((non_speech_normed, speech_normed), dim=0), full_matrices=False, compute_uv=True, hermitian=False)
S